In [3]:
import numpy as np
import pandas as pd
from Utils import *
import re
from tqdm import tqdm
#crawler
import requests
from bs4 import BeautifulSoup

In [4]:
# 讀入官方的data
df = pd.read_csv('./data/tbrain_train_final_0610.csv')  

In [5]:
df[0:10]

,news_ID,hyperlink,content,name
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[]
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[]
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[]
3,4,https://www.chinatimes.com/realtimenews/201909...,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,[]
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[]
5,6,http://www.coolloud.org.tw/node/93356,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區 ### 省略內文 ### 市區拒...,[]
6,7,https://m.ctee.com.tw/livenews/gj/ctee/a076720...,華為創辦人任正非接受英國媒體訪問時表示 ### 省略內文 ### 重申中國和加拿大兩國之間的...,[]
7,8,https://www.mirrormedia.mg/story/20191209edi009/,2019.12.09 08:00「再教育營學員全已結業」新疆自治區主席：他們過上了幸福生活文...,[]
8,9,https://www.chinatimes.com/newspapers/20190827...,交通部觀光局26日公布7月國際旅客情況 ### 省略內文 ### 只能長期培養。,[]
9,10,https://news.cnyes.com/news/id/4365094,0理財基金基金汰弱留強該怎麼做？鉅亨買基金2019/08/06 21:30facebook ...,[]


In [6]:
# 取出網站種類
webKind = df["hyperlink"].str.split('/').str.get(2)
webKind, count = np.unique(webKind, return_counts = True)
dict = {"name": webKind,  
        "counts": count
       }
webKind_df = pd.DataFrame(dict)
webKind_df

,name,counts
0,ccc.technews.tw,10
1,domestic.judicial.gov.tw,30
2,ec.ltn.com.tw,6
3,ent.ltn.com.tw,2
4,estate.ltn.com.tw,1
5,finance.technews.tw,56
6,hk.on.cc,57
7,house.ettoday.net,1
8,m.ctee.com.tw,530
9,m.ltn.com.tw,12


In [19]:
def getContext(url):
    print("==========================")
    print("getContext url: ", url)
    try:
        soup, status_code = getSoup(url)
    except:
        soup = None
        status_code = 404
    contextStr = ""
    if status_code != requests.codes.ok:
        print("status_code: ", status_code, "url: ", url)
        contextStr = str(status_code)
    else:
        data = soup.find_all('p')
        for context in data:
            contextStr += context.text
    return contextStr

In [20]:
# 單獨測試
# 有問題 5
index = 6
url = df["hyperlink"][index]
category = getWebsiteCategory(url)
context = getContext(url) 
print("url :", url, "\ncategory :", category, "\ncontext :", context)

getContext url:  https://m.ctee.com.tw/livenews/gj/ctee/a07672002019081610333753
url : https://m.ctee.com.tw/livenews/gj/ctee/a07672002019081610333753 
category : m.ctee.com.tw 
context :  13:49 財經 | 《半導體》英特爾暫停出貨浪潮 外資憂信驊恐遭波及  13:48 生活 | 振興好運到 刮刮樂「幸運開獎機」「幸運777」歡喜登場  13:41 財經 | 《塑膠股》Q3拚回溫 台塑填息3成8  13:41 國際 | 四大科技巨擘執行長同意赴國會作證  13:38 財經 | 《熱門族群》電商不是吃素的！緊咬振興商機優惠一籮筐  13:24 財經 | 《半導體》Q3旺季看俏 日月光投控飆4月新高 華為創辦人任正非接受英國媒體訪問時表示，他相信英國不會拒絕華為參與建設5G網路。此外，他也堅稱，華為設備並沒有安裝「後門」程式。任正非接受英國天空新聞（Sky News）訪問時指出，英國首相強森（Boris Johnson）是一名果斷且有能力的人，他上任3日後已表明英國應該盡快建設5G網絡，相信英國經過嚴格測試，並以認真態度研究後，不會拒絕華為。他重申，華為的電訊設備並無如外界傳聞般按中國政府要求安裝後門程式，形容華為員工不會做一些令華為「破產、自尋死路」的事。但他承認，美國政府將華為列入貿易黑名單，限制美國企業向華為出口零部件及軟體，對華為造成嚴重損害。任正非強調，自從美國針對華為以來，他從未與中共領導人見面，否則將跌入了美國總統川普的圈套，但說若非對方大肆「宣傳」華為，外界根本不知道華為產品的競爭力。針對加拿大政府應美國要求，逮捕華為副董事長孟晚舟，並可能被引渡到美國，任正非認為，這並非一件小事，但孟晚舟目前情況良好，正在如常生活，重申中國和加拿大兩國之間的政治問題不應將華為甚至其家人牽涉在內。　7月壽險宣告利率出爐，前六大壽險不同調。國泰人壽7月份美元、台幣保單利率雙雙下探，南山人壽也調降台幣利變保單利率，中國人壽、新光人壽、富邦人壽按兵不A2 火線焦點 　新台幣連日大升，使得指標匯銀換匯爆大量。根據國內散戶換匯最多的龍頭銀行台銀統計，近五個交易日以來，台幣匯價勁揚帶動民眾進場買

In [11]:
df_addContext = df.copy()

In [12]:
# 網站分類
df_addContext['webKind'] = df_addContext['hyperlink'].apply(lambda x: getWebsiteCategory(x))

In [25]:
# 原本想用lambda 不過這樣爬蟲會爬太快
#df_addContext['contextCrawler'] = df_addContext['hyperlink'].apply(lambda x: getContext(x))
contextList = []
hyperlinkList = df_addContext['hyperlink']
for index, url in enumerate(tqdm(hyperlinkList)):
    context = getContext(url)
    contextList.append(context)

m/news/id/4426303
 98%|█████████▊| 4920/5023 [1:57:27<02:48,  1.64s/it]==========================
getContext url:  https://www.chinatimes.com/newspapers/20190916000455-260210
getContext url:  https://news.mingpao.com/ins/%e5%9c%8b%e9%9a%9b/article/20191101/s00005/1572574130337/%e7%be%8e%e5%8f%83%e8%ad%b0%e5%93%a1%e6%8f%90%e3%80%8ahong-kong-be-water-act%e3%80%8b-%e8%97%89%e6%b3%95%e6%a1%88%e5%88%b6%e8%a3%81%e4%b8%ad%e6%b8%af%e5%ae%98%e5%93%a1
 98%|█████████▊| 4922/5023 [1:57:40<05:39,  3.36s/it]status_code:  404 url:  https://news.mingpao.com/ins/%e5%9c%8b%e9%9a%9b/article/20191101/s00005/1572574130337/%e7%be%8e%e5%8f%83%e8%ad%b0%e5%93%a1%e6%8f%90%e3%80%8ahong-kong-be-water-act%e3%80%8b-%e8%97%89%e6%b3%95%e6%a1%88%e5%88%b6%e8%a3%81%e4%b8%ad%e6%b8%af%e5%ae%98%e5%93%a1
getContext url:  https://www.chinatimes.com/newspapers/20191117000329-260202
getContext url:  https://sina.com.hk/news/article/20190927/1/11/2/%E4%B8%89%E5%8D%83%E5%81%87%E8%A9%95%E5%88%86%E6%AF%80%E6%8E%89%E4%B8%80%E9%83%A

In [26]:
df_addContext["content_c"] = ""
for index, text in enumerate(tqdm(contextList)):
    df_addContext.at[index, "content_c"] = text

100%|██████████| 5023/5023 [00:00<00:00, 125713.74it/s]


In [27]:
df_addContext.to_csv('./data/Result.csv')